In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Feature Extraction

In [0]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [0]:
def count(X):
  unique_elements, counts_elements = np.unique(X, return_counts=True)
  counter = {}
  for i in range(0,len(unique_elements)):
    counter[unique_elements[i]] = counts_elements[i]
  return counter

In [0]:
# header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
# for i in range(1, 21):
#     header += f' mfcc{i}'
# header += ' label'
# header = header.split()

In [0]:
# #Extractacting Features
# file = open('data.csv', 'w', newline='')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
# digits = '0 1 2 3 4 5 6 7 8 9'.split()
# for g in digits:
#     for filename in os.listdir(f'/content/drive/My Drive/SEM2/DMV/DATASET/Audio/free-spoken-digit-dataset-master/recordings/{g}'):
#         songname = f'/content/drive/My Drive/SEM2/DMV/DATASET/Audio/free-spoken-digit-dataset-master/recordings/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True)
#         chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
#         spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
#         spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
#         rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
#         zcr = librosa.feature.zero_crossing_rate(y)
#         mfcc = librosa.feature.mfcc(y=y, sr=sr)
#         to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
#         for e in mfcc:
#             to_append += f' {np.mean(e)}'
#         to_append += f' {g}'
#         file = open('data.csv', 'a', newline='')
#         with file:
#             writer = csv.writer(file)
#             writer.writerow(to_append.split())

In [0]:
df = pd.read_csv("/content/drive/My Drive/SEM2/DMV/DATASET/Audio/free-spoken-digit-dataset-master/data.csv")
df.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0_jackson_22.wav,0.258648,745.695738,762.953866,1431.958008,0.036321,-316.500122,196.557661,-13.484174,9.429963,15.766304,-32.170087,-16.867396,-15.604297,-23.715188,-0.809440,-11.740212,-21.579213,-0.361823,-9.832786,-15.594936,4.432394,-3.506502,-7.609927,7.188742,-6.255731,0
1,0_theo_47.wav,0.261235,926.016580,971.813501,1970.886230,0.032525,-563.185476,208.009299,-61.913284,22.304171,43.880034,-25.505667,0.460557,-7.064632,-24.361878,10.943825,-8.989138,-25.770058,5.353307,-4.711323,-9.348581,9.180730,-10.025489,-13.303840,1.078143,-14.277954,0
2,0_nicolas_47.wav,0.341204,1343.078468,1163.867228,2842.921143,0.041577,8.738895,236.550249,-102.217295,35.362953,36.254924,-46.430951,25.268305,6.874909,-32.756439,17.478397,-6.241484,-22.029187,15.653694,-8.755220,-10.569900,16.096808,-9.020714,-5.278300,10.477244,-11.571934,0
3,0_nicolas_28.wav,0.347543,1427.998733,1199.233316,2968.645685,0.050049,9.098955,233.608606,-107.258201,36.884075,36.263112,-47.182220,26.769224,4.842184,-34.936805,16.332575,-10.011146,-24.727876,14.809578,-9.979275,-10.753211,15.692811,-8.486928,-1.112172,13.538495,-10.639303,0
4,0_theo_16.wav,0.337563,1045.880035,1016.626613,2308.586040,0.040425,-582.826665,203.866733,-59.192266,32.881939,42.484649,-31.343989,0.747207,-13.676716,-29.231679,14.764896,-8.211393,-21.488277,13.881516,-5.223530,-11.856517,11.216848,-7.919344,-9.444499,2.724871,-16.544174,0


In [0]:
print(count(np.asarray(df["label"])))

{0: 150, 1: 150, 2: 150, 3: 150, 4: 150, 5: 150, 6: 150, 7: 150, 8: 150, 9: 150}


In [0]:
X = df.drop(["label","filename"],axis=1)
X.head()

,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.258648,745.695738,762.953866,1431.958008,0.036321,-316.500122,196.557661,-13.484174,9.429963,15.766304,-32.170087,-16.867396,-15.604297,-23.715188,-0.809440,-11.740212,-21.579213,-0.361823,-9.832786,-15.594936,4.432394,-3.506502,-7.609927,7.188742,-6.255731
1,0.261235,926.016580,971.813501,1970.886230,0.032525,-563.185476,208.009299,-61.913284,22.304171,43.880034,-25.505667,0.460557,-7.064632,-24.361878,10.943825,-8.989138,-25.770058,5.353307,-4.711323,-9.348581,9.180730,-10.025489,-13.303840,1.078143,-14.277954
2,0.341204,1343.078468,1163.867228,2842.921143,0.041577,8.738895,236.550249,-102.217295,35.362953,36.254924,-46.430951,25.268305,6.874909,-32.756439,17.478397,-6.241484,-22.029187,15.653694,-8.755220,-10.569900,16.096808,-9.020714,-5.278300,10.477244,-11.571934
3,0.347543,1427.998733,1199.233316,2968.645685,0.050049,9.098955,233.608606,-107.258201,36.884075,36.263112,-47.182220,26.769224,4.842184,-34.936805,16.332575,-10.011146,-24.727876,14.809578,-9.979275,-10.753211,15.692811,-8.486928,-1.112172,13.538495,-10.639303
4,0.337563,1045.880035,1016.626613,2308.586040,0.040425,-582.826665,203.866733,-59.192266,32.881939,42.484649,-31.343989,0.747207,-13.676716,-29.231679,14.764896,-8.211393,-21.488277,13.881516,-5.223530,-11.856517,11.216848,-7.919344,-9.444499,2.724871,-16.544174


In [0]:
Y = df["label"]

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1, stratify = Y)

# Inbuilt Ensemble Models

 
> **Random Forest**




In [0]:
#GridSearchCV
rf_gsc = GridSearchCV(
        estimator=RandomForestClassifier(),
        param_grid={
            'n_estimators': [50,100,150,200,250,300,350,400,450,500],
            'criterion': ['gini','entropy']
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
rf_grid_result = rf_gsc.fit(X_train, Y_train)
rf_best_params = rf_grid_result.best_params_

In [0]:
print(rf_best_params)

{'criterion': 'gini', 'n_estimators': 150}


In [0]:
cv_model = RandomForestClassifier(n_estimators=450, criterion= 'gini',bootstrap = True, max_features = 'sqrt')
cv_clf = cv_model.fit(X_train, Y_train)

In [0]:
rf_Y_pred = cv_clf.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, rf_Y_pred)
print("Confusion Matrix:\n",confusion_matrix)
from sklearn.metrics import classification_report
print(classification_report(Y_test, rf_Y_pred))

Confusion Matrix:
 [[45  0  0  0  0  0  0  0  0  0]
 [ 0 43  0  1  0  0  0  0  1  0]
 [ 0  0 44  1  0  0  0  0  0  0]
 [ 4  1  1 38  0  0  0  0  0  1]
 [ 0  0  0  0 45  0  0  0  0  0]
 [ 0  0  0  0  0 45  0  0  0  0]
 [ 1  0  0  1  0  0 43  0  0  0]
 [ 0  1  0  0  0  0  2 41  0  1]
 [ 0  3  0  0  0  0  2  0 40  0]
 [ 0  0  0  0  0  0  0  0  1 44]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95        45
           1       0.90      0.96      0.92        45
           2       0.98      0.98      0.98        45
           3       0.93      0.84      0.88        45
           4       1.00      1.00      1.00        45
           5       1.00      1.00      1.00        45
           6       0.91      0.96      0.93        45
           7       1.00      0.91      0.95        45
           8       0.95      0.89      0.92        45
           9       0.96      0.98      0.97        45

    accuracy                           0.95       450




> **ADABoost**



In [0]:
#GridSearchCV
ab_gsc = GridSearchCV(
       estimator=AdaBoostClassifier(),
        param_grid= {
            'n_estimators': [300,350,400,450,500],
            'learning_rate' : [0.8,0.85,0.9,0.95,1]
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
ab_grid_result = ab_gsc.fit(X_train, Y_train)
ab_best_params = ab_grid_result.best_params_
print(ab_best_params)

{'learning_rate': 0.8, 'n_estimators': 500}


In [0]:
cv_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),n_estimators=500,learning_rate=0.8)
cv_clf = cv_model.fit(X_train, Y_train)

In [0]:
ab_Y_pred = cv_clf.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, ab_Y_pred)
print("Confusion Matrix:\n",confusion_matrix)
from sklearn.metrics import classification_report
print(classification_report(Y_test, ab_Y_pred))

Confusion Matrix:
 [[42  0  1  1  0  0  0  1  0  0]
 [ 0 43  0  1  0  0  0  0  1  0]
 [ 1  0 41  3  0  0  0  0  0  0]
 [ 3  1  1 39  0  0  0  0  0  1]
 [ 0  0  0  0 45  0  0  0  0  0]
 [ 0  2  0  0  0 43  0  0  0  0]
 [ 1  0  0  1  0  0 43  0  0  0]
 [ 1  1  0  0  0  0  0 40  0  3]
 [ 0  3  0  0  0  0  0  0 42  0]
 [ 0  0  0  0  0  0  0  0  1 44]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        45
           1       0.86      0.96      0.91        45
           2       0.95      0.91      0.93        45
           3       0.87      0.87      0.87        45
           4       1.00      1.00      1.00        45
           5       1.00      0.96      0.98        45
           6       1.00      0.96      0.98        45
           7       0.98      0.89      0.93        45
           8       0.95      0.93      0.94        45
           9       0.92      0.98      0.95        45

    accuracy                           0.94       450


# Ensemble model with independent models




> **Decision Tree**



In [0]:
#GridSearchCV
dt_gsc = GridSearchCV(
        estimator=DecisionTreeClassifier(),
        param_grid={
            'criterion': ['gini','entropy'],
            'splitter' : ['best','random'],
            'max_depth' : [4,5,6,7,8,9,10,11,15,20],
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
dt_grid_result = dt_gsc.fit(X_train, Y_train)
dt_best_params = dt_grid_result.best_params_
print(dt_best_params)

{'criterion': 'gini', 'max_depth': 15, 'splitter': 'best'}


In [0]:
dt_clf = DecisionTreeClassifier(criterion = "gini", splitter = 'best', max_depth = 15)
dt_clf.fit(X_train, Y_train)
dt_Y_pred = dt_clf.predict(X_test)

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, dt_Y_pred))
print(classification_report(Y_test, dt_Y_pred))

Confusion Matrix:
 [[39  2  1  1  1  0  1  0  0  0]
 [ 3 32  0  2  0  0  0  1  2  5]
 [ 3  2 33  1  0  2  0  1  2  1]
 [ 5  0  1 37  0  0  0  1  0  1]
 [ 6  2  0  0 37  0  0  0  0  0]
 [ 1  3  0  0  0 37  0  1  3  0]
 [ 0  0  2  2  0  0 35  4  2  0]
 [ 0  0  1  0  3  0  2 34  3  2]
 [ 2  1  2  0  1  0  1  2 36  0]
 [ 3  2  2  0  0  2  1  0  1 34]]
              precision    recall  f1-score   support

           0       0.63      0.87      0.73        45
           1       0.73      0.71      0.72        45
           2       0.79      0.73      0.76        45
           3       0.86      0.82      0.84        45
           4       0.88      0.82      0.85        45
           5       0.90      0.82      0.86        45
           6       0.88      0.78      0.82        45
           7       0.77      0.76      0.76        45
           8       0.73      0.80      0.77        45
           9       0.79      0.76      0.77        45

    accuracy                           0.79       450




> **Logistic Regression**



In [0]:
#GridSearchCV
lr_gsc = GridSearchCV(
        estimator=LogisticRegression(),
        param_grid={
            'max_iter' : [250,500,750,1000,1500,2000,3000],
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
lr_grid_result = lr_gsc.fit(X_train, Y_train)
lr_best_params = lr_grid_result.best_params_
print(lr_best_params)

{'max_iter': 3000}


In [0]:
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, Y_train)
lr_Y_pred = lr_clf.predict(X_test)

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, lr_Y_pred))
print(classification_report(Y_test, lr_Y_pred))

Confusion Matrix:
 [[37  2  2  1  0  0  0  2  1  0]
 [ 1 37  0  1  1  0  0  2  0  3]
 [ 1  0 40  2  0  0  1  0  0  1]
 [ 1  0  2 37  0  0  1  1  2  1]
 [ 0  0  0  0 45  0  0  0  0  0]
 [ 0  2  0  0  1 42  0  0  0  0]
 [ 2  0  1  1  0  0 38  1  2  0]
 [ 0  0  0  0  2  2  0 38  0  3]
 [ 0  0  0  1  0  0  0  0 44  0]
 [ 1  1  0  0  0  1  0  1  0 41]]
              precision    recall  f1-score   support

           0       0.86      0.82      0.84        45
           1       0.88      0.82      0.85        45
           2       0.89      0.89      0.89        45
           3       0.86      0.82      0.84        45
           4       0.92      1.00      0.96        45
           5       0.93      0.93      0.93        45
           6       0.95      0.84      0.89        45
           7       0.84      0.84      0.84        45
           8       0.90      0.98      0.94        45
           9       0.84      0.91      0.87        45

    accuracy                           0.89       450




>**SVM**



In [0]:
#GridSearchCV
svm_gsc = GridSearchCV(
        SVC(),
        param_grid={
            'kernel': ['poly','rbf'],
            'gamma' : ['scale','auto'],
            'degree' : [2,3,4],
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
# Feature Scaling
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [0]:
svm_grid_result = svm_gsc.fit(X_train_sc, Y_train)
svm_best_params = svm_grid_result.best_params_
print(svm_best_params)

{'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


In [0]:
svm_clf = SVC(degree= 2, gamma= 'scale', kernel= 'rbf')
svm_clf.fit(X_train_sc, Y_train)
svm_Y_pred = svm_clf.predict(X_test_sc)

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, svm_Y_pred))
print(classification_report(Y_test, svm_Y_pred))

Confusion Matrix:
 [[43  0  1  1  0  0  0  0  0  0]
 [ 1 40  0  1  0  1  0  1  1  0]
 [ 0  0 45  0  0  0  0  0  0  0]
 [ 3  1  1 39  0  1  0  0  0  0]
 [ 0  0  0  0 45  0  0  0  0  0]
 [ 0  0  0  0  0 44  0  1  0  0]
 [ 1  0  0  0  0  0 43  1  0  0]
 [ 1  1  0  0  0  0  0 41  0  2]
 [ 0  3  0  0  0  0  0  0 42  0]
 [ 0  0  0  0  0  0  0  0  1 44]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.91        45
           1       0.89      0.89      0.89        45
           2       0.96      1.00      0.98        45
           3       0.95      0.87      0.91        45
           4       1.00      1.00      1.00        45
           5       0.96      0.98      0.97        45
           6       1.00      0.96      0.98        45
           7       0.93      0.91      0.92        45
           8       0.95      0.93      0.94        45
           9       0.96      0.98      0.97        45

    accuracy                           0.95       450




> **ANN**




In [0]:
from sklearn.preprocessing import StandardScaler
import keras 

Using TensorFlow backend.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [0]:
# Feature Scaling
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [0]:
# initializing the ANN
classifier = Sequential()

#adding input layer and first hidden layer
classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform", input_dim = 25))

#adding second hidden layer
classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform"))

#adding output layer
classifier.add(Dense(activation="softmax", units=10, kernel_initializer="uniform"))

#compiling the ANN
classifier.compile(optimizer = "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

In [0]:
# fitting ANN to the training set
classifier.fit(X_train_sc, Y_train, batch_size = 25, epochs = 500)

#making predictions and evaluating the model
# Predicting probability the Test set results
ANN_prob = classifier.predict(X_test_sc)

Epoch 1/500
42/42 [==============================] - 0s 1ms/step - loss: 2.2898 - accuracy: 0.2038
Epoch 2/500
42/42 [==============================] - 0s 1ms/step - loss: 2.1924 - accuracy: 0.1886
Epoch 3/500
42/42 [==============================] - 0s 1ms/step - loss: 2.0310 - accuracy: 0.1943
Epoch 4/500
42/42 [==============================] - 0s 1ms/step - loss: 1.8809 - accuracy: 0.2152
Epoch 5/500
42/42 [==============================] - 0s 1ms/step - loss: 1.7411 - accuracy: 0.2838
Epoch 6/500
42/42 [==============================] - 0s 1ms/step - loss: 1.5904 - accuracy: 0.3771
Epoch 7/500
42/42 [==============================] - 0s 1ms/step - loss: 1.4376 - accuracy: 0.4771
Epoch 8/500
42/42 [==============================] - 0s 1ms/step - loss: 1.3042 - accuracy: 0.5381
Epoch 9/500
42/42 [==============================] - 0s 1ms/step - loss: 1.2002 - accuracy: 0.5705
Epoch 10/500
42/42 [==============================] - 0s 1ms/step - loss: 1.1201 - accuracy: 0.5886
Epoch 11/

In [0]:
# Getting class values from probabilities
ANN_Y_pred = []
for i in ANN_prob:
  x = i.tolist()
  ANN_Y_pred.append(x.index(max(x)))

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, ANN_Y_pred))
print(classification_report(Y_test, ANN_Y_pred))

Confusion Matrix:
 [[40  2  1  0  0  0  0  1  0  1]
 [ 0 42  0  0  0  0  0  1  2  0]
 [ 0  0 42  3  0  0  0  0  0  0]
 [ 1  2  2 37  1  0  0  1  1  0]
 [ 0  1  0  0 44  0  0  0  0  0]
 [ 0  3  0  0  0 39  0  3  0  0]
 [ 0  0  0  0  1  0 44  0  0  0]
 [ 2  2  0  0  0  0  2 37  1  1]
 [ 0  0  0  0  0  0  2  0 43  0]
 [ 0  1  0  0  0  0  0  1  0 43]]
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        45
           1       0.79      0.93      0.86        45
           2       0.93      0.93      0.93        45
           3       0.93      0.82      0.87        45
           4       0.96      0.98      0.97        45
           5       1.00      0.87      0.93        45
           6       0.92      0.98      0.95        45
           7       0.84      0.82      0.83        45
           8       0.91      0.96      0.93        45
           9       0.96      0.96      0.96        45

    accuracy                           0.91       450


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform", input_dim = 25))
    classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform"))
    classifier.add(Dense(activation="softmax", units=10, kernel_initializer="uniform"))
    classifier.compile(optimizer = 'adam', loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train_sc, y = Y_train, cv = 10, n_jobs = -1)
print(accuracies.mean())
print(accuracies.std())

0.919047623872757
0.018687062572210478


GETTING COMBINED RESULT ACCURACY

In [0]:
combined_Y_pred_list = []
for dt,lr,ann,svm in zip(dt_Y_pred, lr_Y_pred, ANN_Y_pred, svm_Y_pred):
  combined_Y_pred_list.append([dt,lr,ann,svm])

In [0]:
combined_Y_pred = []
for inner_list in combined_Y_pred_list:
  try:
    combined_Y_pred.append(statistics.mode(inner_list))
  except statistics.StatisticsError:
    combined_Y_pred.append(inner_list[3])

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, ANN_Y_pred))
print(classification_report(Y_test, combined_Y_pred))

Confusion Matrix:
 [[40  2  1  0  0  0  0  1  0  1]
 [ 0 42  0  0  0  0  0  1  2  0]
 [ 0  0 42  3  0  0  0  0  0  0]
 [ 1  2  2 37  1  0  0  1  1  0]
 [ 0  1  0  0 44  0  0  0  0  0]
 [ 0  3  0  0  0 39  0  3  0  0]
 [ 0  0  0  0  1  0 44  0  0  0]
 [ 2  2  0  0  0  0  2 37  1  1]
 [ 0  0  0  0  0  0  2  0 43  0]
 [ 0  1  0  0  0  0  0  1  0 43]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        45
           1       0.95      0.89      0.92        45
           2       0.98      1.00      0.99        45
           3       0.95      0.91      0.93        45
           4       1.00      1.00      1.00        45
           5       0.96      0.98      0.97        45
           6       1.00      0.96      0.98        45
           7       0.91      0.91      0.91        45
           8       0.96      1.00      0.98        45
           9       0.96      0.98      0.97        45

    accuracy                           0.96       450


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform", input_dim = 25))
    classifier.add(Dense(activation="relu", units=17, kernel_initializer="uniform"))
    classifier.add(Dense(activation="softmax", units=10, kernel_initializer="uniform"))
    classifier.compile(optimizer = optimizer, loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [250, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train_sc, Y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Streaming output truncated to the last 5000 lines.
Epoch 130/500
945/945 [==============================] - 0s 35us/step - loss: 0.1842 - accuracy: 0.9397
Epoch 131/500
945/945 [==============================] - 0s 35us/step - loss: 0.1821 - accuracy: 0.9407
Epoch 132/500
945/945 [==============================] - 0s 33us/step - loss: 0.1796 - accuracy: 0.9386
Epoch 133/500
945/945 [==============================] - 0s 36us/step - loss: 0.1790 - accuracy: 0.9397
Epoch 134/500
945/945 [==============================] - 0s 36us/step - loss: 0.1790 - accuracy: 0.9386
Epoch 135/500
945/945 [==============================] - 0s 37us/step - loss: 0.1759 - accuracy: 0.9407
Epoch 136/500
945/945 [==============================] - 0s 39us/step - loss: 0.1747 - accuracy: 0.9418
Epoch 137/500
945/945 [==============================] - 0s 37us/step - loss: 0.1728 - accuracy: 0.9481
Epoch 138/500
945/945 [==============================] - 0s 35us/step - loss: 0.1707 - accuracy: 0.9418
Epoch 139/500

**CONCLUSION**

**MODEL**                              **ACCURACY**

Logistic regression--                        89%

Decision Tree  --                           79%

ANN        --                               91%

SVM   --                                    95%

Random Forest Algorithm   --                95%

Adaboost Algorithm            --            95%

Bagging(Combination of all)   --             97%

